In [423]:
import numpy as np
from datetime import date
from dateutil.relativedelta import relativedelta
import abc

# QuantLib imports
import QuantLib as ql

class G2Calibrator:
    """
    Calibrates a G2++ model to market cap/floor quotes.
    """
    def __init__(self, ts_handle, index):
        self.ts_handle = ts_handle
        self.index = index
        self.model = ql.G2(ts_handle)

    def calibrate(
        self,
        periods: list,
        quotes: list,
        optimization_method=None,
        end_criteria=None,
        engine_steps: int = 50
    ) -> tuple:
        """
        Calibrate the G2 model to cap vols.

        Args:
            periods: list of ql.Period objects (e.g. [ql.Period('1Y'),...])
            quotes:  list of QuoteHandle objects matching periods
            optimization_method: ql.LevenbergMarquardt instance
            end_criteria: ql.EndCriteria instance
            engine_steps: steps for TreeCapFloorEngine

        Returns:
            tuple of calibrated (a, sigma, b, eta, rho)
        """
        # build helpers
        helpers = []
        for p, q in zip(periods, quotes):
            # CapHelper with explicit errorType, volType, shift
            helper = ql.CapHelper(
                p,
                q,
                self.index,
                ql.Quarterly,
                ql.Actual360(),
                False,
                self.ts_handle,
                ql.BlackCalibrationHelper.RelativePriceError,
                ql.Normal,
                0.0
            )
            engine = ql.TreeCapFloorEngine(self.model, engine_steps)
            helper.setPricingEngine(engine)
            helpers.append(helper)

        # default optimization and end criteria
        opt = optimization_method or ql.LevenbergMarquardt(1e-8,1e-8,1e-8)
        criteria = end_criteria or ql.EndCriteria(1000,500,1e-8,1e-8,1e-8)

        # calibrate
        self.model.calibrate(helpers, opt, criteria)
        return self.model.params()


class QuantLibBondStaticBase:
    """
    Encapsulates static bond definition in QuantLib: schedule, instrument, and evaluation-date setup.
    """
    def __init__(
        self,
        valuation_date: date,
        maturity_date: date,
        coupon_rate: float,
        face_value: float = 100.0,
        freq: int = 2,
        calendar=None,
        day_count=None,
        business_convention=None,
    ):
        # Store Python-side
        self.valuation_date_py = valuation_date
        self.maturity_date_py = maturity_date
        self.coupon_rate = coupon_rate
        self.face_value = face_value
        self.freq = freq

        # QuantLib settings (defaults if None)
        self.calendar = calendar or ql.NullCalendar()
        self.day_count = day_count or ql.Actual365Fixed()
        self.business_convention = (
            business_convention if business_convention is not None else ql.Unadjusted
        )

        # Convert to QL Dates and set global evaluation date
        self.ql_valuation_date = ql.Date(
            valuation_date.day, valuation_date.month, valuation_date.year
        )
        ql.Settings.instance().evaluationDate = self.ql_valuation_date
        self.ql_maturity_date = ql.Date(
            maturity_date.day, maturity_date.month, maturity_date.year
        )

        # Build payment schedule
        months = int(12 / self.freq)
        self.schedule = ql.Schedule(
            self.ql_valuation_date,
            self.ql_maturity_date,
            ql.Period(months, ql.Months),
            self.calendar,
            self.business_convention,
            self.business_convention,
            ql.DateGeneration.Forward,
            False,
        )

        # Build the FixedRateBond instrument
        self.bond = ql.FixedRateBond(
            0,
            self.face_value,
            self.schedule,
            [self.coupon_rate],
            self.day_count,
        )

class CallableBondStaticBase(QuantLibBondStaticBase):
    """
    Extends QuantLibBondStaticBase to add a call schedule and build
    a CallableFixedRateBond.
    """
    def __init__(
        self,
        valuation_date: date,
        maturity_date: date,
        coupon_rate: float,
        call_dates: list[date],
        call_prices: list[float],
        face_value: float = 100.0,
        freq: int = 2,
        calendar=None,
        day_count=None,
        business_convention=None,
    ):
        # initialize the plain vanilla bond
        super().__init__(
            valuation_date, maturity_date,
            coupon_rate, face_value, freq,
            calendar, day_count, business_convention
        )

        # build a callability schedule
        self.call_schedule = ql.CallabilitySchedule()
        for cd, cp in zip(call_dates, call_prices):
            ql_cd = ql.Date(cd.day, cd.month, cd.year)
            callability_price  = ql.BondPrice(cp, ql.BondPrice.Clean)
            call = ql.Callability(
                callability_price,
                ql.Callability.Call,
                ql_cd
            )
            self.call_schedule.push_back(call)

        # rebuild bond as CallableFixedRateBond
        settlement_days = 0
        self.bond = ql.CallableFixedRateBond(
            settlement_days,
            self.face_value,
            self.schedule,
            [self.coupon_rate],
            self.day_count,
            self.business_convention,
            self.face_value,
            self.ql_valuation_date,
            self.call_schedule
        )

class BondPricerBase(abc.ABC):
    """
    Abstract pricer interface. Operates on a static bond definition.
    """
    def __init__(self, bond_static: QuantLibBondStaticBase):
        self.bond_static = bond_static

    @abc.abstractmethod
    def price(
        self,
        pillar_times: np.ndarray,
        zero_rates: np.ndarray,
    ) -> np.ndarray:
        pass

class FastBondPricer(BondPricerBase):
    """
    Fast numpy-based pricer (Actual/365Fixed) with interpolation.
    """
    def __init__(self, bond_static: QuantLibBondStaticBase):
        super().__init__(bond_static)
        self._gen_cashflows()

    def _gen_cashflows(self):
        ql_sched = self.bond_static.schedule
        dc       = self.bond_static.day_count
        ql_val   = self.bond_static.ql_valuation_date

        cf_dates   = []
        cf_times   = []
        cf_amts    = []
        coupon_amt = self.bond_static.face_value * self.bond_static.coupon_rate / self.bond_static.freq

        for d in ql_sched:
            if d == ql_val:
                continue
            cf_dates.append(d.to_date())
            t = dc.yearFraction(ql_val, d)
            cf_times.append(t)
            cf_amts.append(coupon_amt)

        cf_amts[-1] += self.bond_static.face_value

        self.cf_dates   = cf_dates
        self.cf_times   = np.array(cf_times, dtype=float)
        self.cf_amounts = np.array(cf_amts,  dtype=float)

    def price(self, pillar_times: np.ndarray, zero_rates: np.ndarray) -> np.ndarray:
        if zero_rates.ndim == 1:
            r_cf = np.interp(self.cf_times, pillar_times, zero_rates)
            dfs  = np.exp(-r_cf * self.cf_times)
            return float(self.cf_amounts.dot(dfs))
        r_cf_matrix = np.vstack([
            np.interp(self.cf_times, pillar_times, sc)
            for sc in zero_rates
        ])
        dfs = np.exp(-r_cf_matrix * self.cf_times[None, :])
        return dfs.dot(self.cf_amounts)

class QuantLibBondPricer(BondPricerBase):
    """
    Pricer using QuantLib's time-based ZeroCurve + appropriate engine.
    - Uses DiscountingBondEngine for vanilla bonds.
    - Uses TreeCallableFixedRateBondEngine with G2 model for callable bonds (method='g2').

    Args:
        bond_static: static bond definition (vanilla or callable)
        method: 'discount' or 'g2'
        grid_steps: time steps for tree engine (if callable)
    """
    def __init__(self, bond_static: QuantLibBondStaticBase, method: str = 'discount', grid_steps: int = 100):
        super().__init__(bond_static)
        self.method = method.lower()
        self.grid_steps = grid_steps

    def _make_term_structure(self, pillar_times, rates_vec):
        """
        Build a date-based zero curve from float year-times and rates.
        """
        # anchor at valuation date
        base = self.bond_static.ql_valuation_date
        # build date vector: time zero plus each pillar
        dates = ql.DateVector()
        dates.push_back(base)
        for t in pillar_times:
            days = int(round(t * 365))
            dates.push_back(base + ql.Period(days, ql.Days))
        # prepend zero-time rate
        rates = [rates_vec[0]] + list(rates_vec)
        # build zero curve
        zc = ql.ZeroCurve(
            dates,
            rates,
            self.bond_static.day_count,
            self.bond_static.calendar,
            ql.Linear(),
            ql.Continuous,
            ql.Annual
        )
        zc.enableExtrapolation()
        return ql.YieldTermStructureHandle(zc)

    @staticmethod
    def _price_vanilla(bond, ts_handle):
        engine = ql.DiscountingBondEngine(ts_handle)
        bond.setPricingEngine(engine)
        return bond.NPV()

    @staticmethod
    def _price_callable(bond, ts_handle, model_params, grid_steps):
        # model_params: tuple (a, sigma, b, eta, rho)
        a, sigma, b, eta, rho = model_params
        model = ql.G2(ts_handle, a, sigma, b, eta, rho)
        engine = ql.TreeCallableFixedRateBondEngine(model, grid_steps)
        bond.setPricingEngine(engine)
        return bond.cleanPrice()

    def price(
        self,
        pillar_times: np.ndarray,
        zero_rates: np.ndarray,
        g2_params=None
    ) -> np.ndarray:
        """
        For method='g2', supply g2_params:
          - single tuple (a,sigma,b,eta,rho) to apply to all curves,
          - or iterable of tuples matching zero_rates.shape[0]
        """
        dc  = self.bond_static.day_count
        cal = self.bond_static.calendar
        is_callable = hasattr(self.bond_static, 'call_schedule') and self.method == 'g2'

        def price_curve(rates_vec, params=None):
            ts = self._make_term_structure(pillar_times, rates_vec)
            if is_callable:
                return self._price_callable(
                    self.bond_static.bond,
                    ts,
                    params if params is not None else g2_params,
                    self.grid_steps
                )
            else:
                return self._price_vanilla(
                    self.bond_static.bond,
                    ts
                )

        # single curve
        if zero_rates.ndim == 1:
            return float(price_curve(zero_rates, g2_params))

        # multi-scenario
        prices = []
        # if single params tuple, broadcast
        single_params = g2_params and not hasattr(g2_params[0], '__iter__')
        for idx, scen in enumerate(zero_rates):
            params = g2_params if single_params else (g2_params[idx] if g2_params else None)
            prices.append(price_curve(scen, params))
        return np.array(prices)


class TensorFunctionalForm:
    """
    Represents a multivariate quadratic function f(x) = x^T A x + b^T x + c,
    where A is a full symmetric matrix of second-order coefficients.
    Supports both single-vector and matrix inputs for efficient pricing.
    """
    def __init__(self, A: np.ndarray, b: np.ndarray, c: float):
        # A is a (D,D) symmetric matrix, b is vector length D
        self.A = A
        self.b = b
        self.c = c

    def __call__(self, x: np.ndarray) -> np.ndarray:
        """
        Evaluate the quadratic form: f(x) = x^T A x + b^T x + c.
        Args:
            x: np.ndarray shape (D,) or (D,N) or (N,D)
        Returns:
            scalar or np.ndarray of shape (D,)
        """
        x_arr = np.asarray(x)
        if x_arr.ndim == 1:
            # single scenario vector of length D
            return float(x_arr @ (self.A.dot(x_arr)) + self.b.dot(x_arr) + self.c)
        elif x_arr.ndim == 2:
            # apply quadratic form at the same time without for loop
            # check if the shape is N x D or D x N
            if x_arr.shape[1] == self.A.shape[0]:
                # N x D
                return np.einsum('ij,jk,ik->i', x_arr, self.A, x_arr) + x_arr @ self.b + self.c
            elif x_arr.shape[0] == self.A.shape[0]:
                # D x N
                # transpose to N x D
                x_arr = x_arr.T
                return np.einsum('ij,jk,ik->i', x_arr, self.A, x_arr) + x_arr @ self.b + self.c
            else:
                raise ValueError(f"Invalid input shape {x_arr.shape} for quadratic form.")
           

        else:
            raise ValueError(f"Input must be 1D or 2D array, got ndim={x_arr.ndim}")
        return None  # unreachable, but for type hinting
    
        
class TensorFunctionalFormCalibrate:
    """
    Builds and evaluates a TensorFunctionalForm approximation of bond prices
    over multi-dimensional zero-rate shocks.

    Usage requirements:
      - For a vanilla pricer (FastBondPricer or QuantLibBondPricer(method='discount')):
          call sample_and_fit(simulated_curves, pillar_times, ...)
      - For a callable pricer with G2 model:
          call sample_and_fit(simulated_curves, pillar_times, g2_params=..., ...)
      - Additional keyword args forwarded to pricer.price
    """
    
    def __init__(
        self,
        pricer,                # instance of BondPricerBase
        pillar_times: np.ndarray,
        base_curve: np.ndarray  # base zero curve
    ):
        self.pricer = pricer
        self.pillar_times = pillar_times
        self.base_curve = base_curve
        
    def sample_and_fit(
        self,
        full_scenarios: np.ndarray,
        *price_args,
        n_train: int = 50,
        n_test: int = 20,
        random_seed: int = 0,
        **price_kwargs
    ) -> tuple[TensorFunctionalForm, np.ndarray, np.ndarray, float]:
        """
        Samples training and testing scenarios and fits a TensorFunctionalForm.

        Arguments:
          full_scenarios: np.ndarray, shape (N_full, D)
            Matrix of zero-rate curves used to define sampling domain and test set.

        Positional args (*price_args):
          Arguments for pricer.price before the scenario matrix, e.g.:
            - pillar_times: np.ndarray of floats (M,), required for interpolation.

        Keyword args (**price_kwargs):
          Additional args for pricer.price, e.g.:
            - g2_params: tuple or sequence of tuples for callable G2 pricing.

        Optional sampling parameters:
          n_train: number of training points sampled uniformly.
          n_test:  number of testing points selected from full_scenarios.
          random_seed: seed for reproducibility.

        Returns:
          model: TensorFunctionalForm fitted to training data.
          test_scen: np.ndarray of test scenarios.
          test_prices: actual pricer prices on test scenarios.
          rmse: root mean squared error between model and actual prices.
        """
        rng = np.random.default_rng(random_seed)
        # define domain bounds from full_scenarios
        domain_min = np.min(full_scenarios, axis=0)
        domain_max = np.max(full_scenarios, axis=0)
        D = full_scenarios.shape[1]

        # 1) Uniform sampling over [min, max] in each dimension
        train_scen = rng.uniform(
            low=domain_min,
            high=domain_max,
            size=(n_train, D)
        )

        # 2) Price training scenarios
        train_prices = self.pricer.price(*price_args, train_scen, **price_kwargs)

        # 3) Fit tensor quadratic with cross terms
        X_train = np.hstack([
            np.einsum('ij,ik->ijk', train_scen, train_scen).reshape(n_train, -1),  # quadratic terms
            train_scen,  # linear terms
            np.ones((n_train, 1))  # constant term
        ])
        coeffs, *_ = np.linalg.lstsq(X_train, train_prices, rcond=None)
        
        # Extract coefficients
        # coeffs: [a_ij, b_i, c]
        a = coeffs[:-D-1].reshape(D, D)  # quadratic terms
        b = coeffs[-D-1:-1]  # linear terms
        c = coeffs[-1]  # constant term
        
        model = TensorFunctionalForm(a, b, c)

        # 4) Test on random subset of full_scenarios
        idx = rng.choice(full_scenarios.shape[0], size=n_test, replace=False)
        test_scen = full_scenarios[idx]
        test_true = self.pricer.price(*price_args, test_scen, **price_kwargs)
        
        test_pred = model(test_scen)
        #test_pred = np.array([model(x) for x in test_scen])

        # 5) compute RMSE
        rmse = np.sqrt(np.mean((test_true - test_pred)**2))
        return model, test_scen, test_true, rmse


# -------------------------------------------------------------------
# Example usage in test harness
# -------------------------------------------------------------------
import time

if __name__ == "__main__":
    val_date  = date(2025, 5, 18)
    mat_date  = date(2030, 5, 18)
    coupon    = 0.03
    
    # Callable static
    call_dates  = [date(2027,5,18), date(2028,5,18), date(2029,5,18)]
    call_prices = [110.0, 110.0, 110.0]
    cb_static   = CallableBondStaticBase(
        val_date, mat_date, coupon,
        call_dates, call_prices
    )

    # Plain vanilla
    bond_static = QuantLibBondStaticBase(val_date, mat_date, coupon)
    fast_pricer = FastBondPricer(bond_static)
    ql_pricer   = QuantLibBondPricer(bond_static)
    ql_callable_pricer = QuantLibBondPricer(cb_static, method='g2', grid_steps=10)
    
    
    # Market curve - single curve and simulated scenarios
    pillar_times = np.array([1,2,3,5,7,10], dtype=float)
    zero_curve   = np.array([0.02,0.021,0.022,0.025,0.027,0.03], dtype=float)

    # Generate multiple simulated zero-curve scenarios
    np.random.seed(42)
    n_scenarios = 2000
    
    # This is a simulated zero curve with a small random perturbation - not a realistic model
    simulated_curves = zero_curve[None, :] + 0.001 * np.random.randn(n_scenarios, zero_curve.size)

    print("Plain vanilla single-curve price:")
    print(" Fast  :", fast_pricer.price(pillar_times, zero_curve))
    print(" QL    :", ql_pricer.price(pillar_times, zero_curve))

    # --------------- Model-based callable pricer ---------------
    print("Callable bond pricing via calibrated G2++ Model:")
    
    # Use the pricer's term-structure builder to avoid manual ZeroCurve call
    pricer = QuantLibBondPricer(bond_static)  # Define the pricer instance
    ts_handle = pricer._make_term_structure(pillar_times, zero_curve)

    # Build Libor index for calibration
    index = ql.Sofr(ts_handle)

    # Market cap vol quotes for G2 calibration
    periods = [ql.Period('1Y'), ql.Period('2Y'), ql.Period('5Y'), ql.Period('7Y'), ql.Period('10Y'), ql.Period('15Y')]
    vols    = [0.0185,        0.0155,         0.0145,         0.0143,         0.0135,          0.0125]
    quotes  = [ql.QuoteHandle(ql.SimpleQuote(v)) for v in vols]

    # Calibrate G2 model
    calibrator = G2Calibrator(ts_handle, index)

    g2_params = [ 0.0937371, 0.0201141, 1.14135, 0.046775, -0.886926 ]
    print(" Calibrated G2 parameters:", g2_params)

    # Price callable bond using calibrated parameters
    model_price = ql_callable_pricer.price(pillar_times, simulated_curves, g2_params=g2_params)
    print(" G2++ Callable Price (calibrated):", model_price)
  
    # --------------- Tensor approximation example for callable bond ---------------
    print("\nFitting tensor approximation for vanilla bond price:")
    # Build a full scenario set around the base zero curve (e.g. simulated_curves above)
    tf_calibrator_vanilla = TensorFunctionalFormCalibrate(ql_pricer, pillar_times, simulated_curves)
    
    model_c, test_scen_c, true_prices_c, rmse_c = tf_calibrator_vanilla.sample_and_fit(
        simulated_curves,
        pillar_times,
        n_train=200,
        n_test=10,
        random_seed=43
    )
    print(" Vanilla RMSE:", rmse_c)
    print(" Test true vs approx (5 samples):")
    for tp, ts in zip(true_prices_c[:5], [model_c(x) for x in test_scen_c[:5]]):
        print(f"  true={tp:.4f}, approx={ts:.4f}")
    
    # Build a full scenario set around the base zero curve (e.g. simulated_curves above)
    print("\nFitting tensor approximation for callable bond price:")
        
    start_time = time.time()
    tf_calibrator = TensorFunctionalFormCalibrate(ql_callable_pricer, pillar_times, simulated_curves)
    model_c, test_scen_c, true_prices_c, rmse_c = tf_calibrator.sample_and_fit(
        simulated_curves,
        pillar_times,
        g2_params=g2_params,
        n_train=100,
        n_test=20,
        random_seed=43
    )
    
    calibrate_time = time.time() - start_time
    print(f"calibrate time: {calibrate_time:.4f} seconds")       
    
    print(" Callable RMSE:", rmse_c)
    print(" Test true vs approx (5 samples):")
    for tp, ts in zip(true_prices_c[:5], [model_c(x) for x in test_scen_c[:5]]):
        print(f"  true={tp:.4f}, approx={ts:.4f}")


Plain vanilla single-curve price:
 Fast  : 102.33441524411462
 QL    : 102.34283782186014
Callable bond pricing via calibrated G2++ Model:
 Calibrated G2 parameters: [0.0937371, 0.0201141, 1.14135, 0.046775, -0.886926]
 G2++ Callable Price (calibrated): [101.92078532 102.17035399 102.81661886 ... 102.34489532 102.86490601
 102.25051076]

Fitting tensor approximation for vanilla bond price:
 Vanilla RMSE: 1.134814706572795e-05
 Test true vs approx (5 samples):
  true=103.2964, approx=103.2964
  true=102.1235, approx=102.1235
  true=102.9368, approx=102.9368
  true=102.4141, approx=102.4141
  true=102.7171, approx=102.7171

Fitting tensor approximation for callable bond price:
calibrate time: 0.0146 seconds
 Callable RMSE: 0.0036088103860763203
 Test true vs approx (5 samples):
  true=102.4577, approx=102.4559
  true=103.0670, approx=103.0598
  true=102.3195, approx=102.3210
  true=102.3316, approx=102.3326
  true=102.3262, approx=102.3277


In [424]:
import time

# Generate scenarios
scenarios_1m = zero_curve + 0.001 * np.random.randn(2_000, pillar_times.size)

# Measure time for FastBondPricer
start_time = time.time()
fp_scen_1m = fast_pricer.price(pillar_times, scenarios_1m)
fast_pricer_time_1m = time.time() - start_time
print(f"FastBondPricer time: {fast_pricer_time_1m:.4f} seconds")

# Measure time for TensorFunctionalForm
start_time = time.time()
tf_calibrator = TensorFunctionalFormCalibrate(ql_callable_pricer, pillar_times, simulated_curves)
model_c, test_scen_c, true_prices_c, rmse_c = tf_calibrator.sample_and_fit(
    simulated_curves,
    pillar_times,
    g2_params=g2_params,
    n_train=200,
    n_test=20,
    random_seed=43
)
tf_scen_1m = model_c(scenarios_1m)
tensor_pricer_time_1m = time.time() - start_time
print(f"TensorFunctionalForm + calibration time: {tensor_pricer_time_1m:.4f} seconds")

# Measure time for QuantLibBondPricer
start_time = time.time()
qp_scen_1m = ql_pricer.price(pillar_times, scenarios_1m)
quantlib_pricer_time_1m = time.time() - start_time
print(f"QuantLibBondPricer time: {quantlib_pricer_time_1m:.4f} seconds")

# Measure time TensorFunctionalForm for vanilla bond
start_time = time.time()
tf_calibrator_vanilla = TensorFunctionalFormCalibrate(ql_pricer, pillar_times, simulated_curves)
model_v, test_scen_v, true_prices_v, rmse_v = tf_calibrator_vanilla.sample_and_fit(
    simulated_curves,
    pillar_times,
    n_train=200,
    n_test=10,
    random_seed=43
)
tensor_pricer_vanilla_time = time.time() - start_time
print(f"TensorFunctionalForm + calibration time (vanilla): {tensor_pricer_vanilla_time:.4f} seconds")

# Measure time for CallableBondPricer
start_time = time.time()
cb_scen_1m = ql_callable_pricer.price(pillar_times, scenarios_1m, g2_params=g2_params)
callable_pricer_time_1m = time.time() - start_time
print(f"CallableBondPricer time for: {callable_pricer_time_1m:.4f} seconds")
print()
print(f"Speedup of FastBondPricer over QuantLibBondPricer: {quantlib_pricer_time_1m / fast_pricer_time_1m:.2f}x")
print(f"TensorFunctionalForm speedup over QuantLibBondPricer Vanilla: {quantlib_pricer_time_1m / tensor_pricer_vanilla_time:.2f}x")
print(f"Speedup of TensorFunctionalForm over CallableBondPricer: {callable_pricer_time_1m / tensor_pricer_time_1m:.2f}x")

FastBondPricer time: 0.0047 seconds
TensorFunctionalForm + calibration time: 0.0349 seconds
QuantLibBondPricer time: 0.0526 seconds
TensorFunctionalForm + calibration time (vanilla): 0.0063 seconds
CallableBondPricer time for: 0.2380 seconds

Speedup of FastBondPricer over QuantLibBondPricer: 11.25x
TensorFunctionalForm speedup over QuantLibBondPricer Vanilla: 8.39x
Speedup of TensorFunctionalForm over CallableBondPricer: 6.82x


In [357]:
def generate_bond_collections(
    num_bonds: int,
    valuation_date: date = date(2025, 1, 1),
    face_value: float = 100.0,
    seed: int = 0
):
    """
    Generator of random vanilla and callable bond static definitions for testing aggregation.

    Args:
        num_bonds:      number of bond instances to generate
        valuation_date: common valuation date for all bonds
        face_value:     default face value for bonds
        seed:           random seed for reproducibility

    Returns:
        vanilla_bonds:   list of QuantLibBondStaticBase instances
        callable_bonds:  list of CallableBondStaticBase instances
    """
    rng = np.random.default_rng(seed)
    vanilla_bonds = []
    callable_bonds = []
    for _ in range(num_bonds):
        # random maturity between 2 and 10 years
        years = int(rng.integers(2, 11))
        mat_date = valuation_date + relativedelta(years=years)
        # random coupon rate between 1% and 8%
        coupon = float(rng.uniform(0.01, 0.08))
        # random frequency from annual, semi, quarterly
        freq = int(rng.choice([1, 2, 4]))

        # create vanilla bond static
        vb = QuantLibBondStaticBase(
            valuation_date,
            mat_date,
            coupon,
            face_value=face_value,
            freq=freq
        )
        vanilla_bonds.append(vb)

        # create callable bond static
        # generate between 1 and (years) call dates
        num_calls = int(rng.integers(1, years))
        call_dates = [
            valuation_date + relativedelta(years=j+1)
            for j in range(num_calls)
        ]
        # random call prices around par
        call_prices = [
            float(face_value + rng.uniform(-2.0, 2.0))
            for _ in range(num_calls)
        ]
        cb = CallableBondStaticBase(
            valuation_date,
            mat_date,
            coupon,
            call_dates,
            call_prices,
            face_value=face_value,
            freq=freq
        )
        callable_bonds.append(cb)

    return vanilla_bonds, callable_bonds

In [399]:
vanilla_list, callable_list = generate_bond_collections(100, seed=42)
scenarios = zero_curve + 0.001 * np.random.randn(2000, pillar_times.size)

In [400]:
# measure memory usage
import tracemalloc
tracemalloc.start()
start_time = time.time()
# create pricers
vanilla_pricers = [QuantLibBondPricer(vb) for vb in vanilla_list]

# price vanilla bonds
vanilla_prices = np.array([p.price(pillar_times, scenarios) for p in vanilla_pricers])
# measure memory usage
current, peak = tracemalloc.get_traced_memory()
tracemalloc.stop()
time_taken = time.time() - start_time
print(f"Vanilla bonds pricing time: {time_taken:.4f} seconds")
print(f"Memory usage: {current / 1e6:.2f} MB, Peak: {peak / 1e6:.2f} MB")

Vanilla bonds pricing time: 24.3545 seconds
Memory usage: 1.65 MB, Peak: 3.27 MB


In [404]:
# measure memory usage
tracemalloc.start()
start_time = time.time()
# create pricers
vanilla_pricers = [QuantLibBondPricer(vb) for vb in vanilla_list]
# now create TFF for each pricer and fit them
tff_vanilla = []
for vb, pricer in zip(vanilla_list, vanilla_pricers):
    tff = TensorFunctionalFormCalibrate(pricer, pillar_times, scenarios)
    model, test_scen_c, true_prices_c, rmse_c = tff.sample_and_fit(
        scenarios,
        pillar_times,
        n_train=100,
        n_test=10,
        random_seed=43
    )
    tff_vanilla.append(model)
    
# price vanilla bonds using TFF
tff_prices = np.array([model(scenarios) for model in (tff_vanilla)])
# measure memory usage
current, peak = tracemalloc.get_traced_memory()
tracemalloc.stop()
time_taken = time.time() - start_time
print(f"Vanilla bonds pricing time: {time_taken:.4f} seconds")
print(f"Memory usage: {current / 1e6:.2f} MB, Peak: {peak / 1e6:.2f} MB")

Vanilla bonds pricing time: 1.4785 seconds
Memory usage: 1.75 MB, Peak: 3.36 MB


In [405]:
# compare TFF prices to QuantLib prices as a sum
tff_prices_sum = np.sum(tff_prices)
vanilla_prices_sum = np.sum(vanilla_prices)
tff_prices_sum, vanilla_prices_sum
#print error as a percentage of the sum of the prices
error = np.abs(tff_prices_sum - vanilla_prices_sum) / vanilla_prices_sum * 100
print(f"Error between TFF and QuantLib prices: {error:.4f}%")

Error between TFF and QuantLib prices: 0.0000%


In [421]:
# measure memory usage
tracemalloc.start()
start_time = time.time()

# price each callable bond using the callable pricer
callable_pricers = [QuantLibBondPricer(cb, method='g2', grid_steps=10) for cb in callable_list]

#cr3eate a TFF for each callable bond
tff_callable = []
for cb, pricer in zip(callable_list, callable_pricers):
    tff = TensorFunctionalFormCalibrate(pricer, pillar_times, scenarios)
    model, test_scen_c, true_prices_c, rmse_c = tff.sample_and_fit(
        scenarios,
        pillar_times,
        g2_params=g2_params,
        n_train=100,
        n_test=10,
        random_seed=43
    )
    tff_callable.append(model)
    
# price callable bonds using TFF
tff_prices = np.array([model(scenarios) for model in (tff_callable)])

# measure memory usage
current, peak = tracemalloc.get_traced_memory()
tracemalloc.stop()
time_taken_tff = time.time() - start_time
print(f"TFF Callable bonds pricing time: {time_taken_tff:.4f} seconds")


TFF Callable bonds pricing time: 5.5757 seconds


In [419]:
# measure memory usage
tracemalloc.start()
start_time = time.time()

# price callable bonds
callable_prices = np.array([p.price(pillar_times, scenarios, g2_params=g2_params) for p in callable_pricers])

# measure memory usage
current, peak = tracemalloc.get_traced_memory()
tracemalloc.stop()
time_taken = time.time() - start_time
print(f"Callable bonds pricing time: {time_taken:.4f} seconds")


Callable bonds pricing time: 99.0831 seconds


In [420]:
# compare TFF prices to QuantLib prices as a sum
tff_prices_sum = np.sum(tff_prices)
callable_prices_sum = np.sum(callable_prices)
tff_prices_sum, callable_prices_sum

#print error as a percentage of the sum of the prices
error = np.abs(tff_prices_sum - callable_prices_sum) / callable_prices_sum * 100
print(f"Error between TFF and QuantLib prices: {error:.4f}%")

Error between TFF and QuantLib prices: 0.0003%


In [422]:
print(f"Speedup of TFF over QuantLib for callable bonds: {time_taken / time_taken_tff:.2f}x")

Speedup of TFF over QuantLib for callable bonds: 17.77x
